# Import Externally Fine-Tuned Models into AI Quick Actions environment

This is a guide to import externally fine-tuned model into AI Quick Actions environment. If you already have a model that was fine-tuned in a different environment, this guide will cover the steps required to import it into AI Quick Actions.

- Set up required variables for creating resources
- Identify the existing base model OCID
- Get the base model metadata to pass on to the fine-tuned model
- Create a new Model Version Set if not already present
- Build fine-tuned model custom metadata
- Add optional custom metadata for showing training and validation metrics in AI Quick Actions UI.
- Create a new model catalog entry for fine-tuned model

Once this model is created, you can go to the AI Quick Actions UI where you'll find this model in the `Fine-Tuned models` tab on the `Model Explorer` page. From there, you can continue to deploy the model on OCI Data Science environment.

### Setup

In [ ]:
import ads
ads.set_auth("resource_principal")

In [ ]:
import os
os.environ["ADS_AQUA_LOG_LEVEL"] = "ERROR"

from ads.config import PROJECT_OCID, COMPARTMENT_OCID

# Replace project_id and compartment_id if needed, by default this picks up the compartment and project id of the current notebook session.
project_id = PROJECT_OCID
compartment_id = COMPARTMENT_OCID

### Find base model

In [ ]:
# Search keyword to look up base models in AI Quick Actions, used to find and filter the model we're trying to include along with the fine-tuned model.
MODEL_SEARCH_KEYWORD = "mistral" # <-- Change this for your use case

In [ ]:
from ads.aqua.model import AquaModelApp

In [ ]:
model_list = []
service_model_list = AquaModelApp().list()
registered_model_list = AquaModelApp().list(compartment_id=compartment_id, model_type="BASE")

model_list.extend(service_model_list)
model_list.extend(registered_model_list)

for model in model_list:
    if MODEL_SEARCH_KEYWORD.lower() in model.name.lower():
        print(f"Model: {model.name}, OCID: {model.id}, Registration Required: {'Yes' if model.ready_to_import else 'No'}")

In the above results, if **Registration Required** is "Yes", then go to AI Quick Actions UI to register the model and import the base model artifacts to object storage. If **Registration Required** is "No", you can proceed to the next step as artifacts are already included with the model.

Once you identify the model to use, copy the model OCID and set it in the next block as `source_model_ocid`.

### Load source model

In [ ]:
from ads.model import DataScienceModel
from ads.aqua.common.utils import get_artifact_path

source_model_ocid = "ocid1.datasciencemodel.oc1.<region>.<ocid>" # <-- Change this OCID based on model to use as base model
ds_model = DataScienceModel.from_id(source_model_ocid)

In [ ]:
source_artifact_location = get_artifact_path(ds_model.custom_metadata_list._to_oci_metadata())
deployment_container = ds_model.custom_metadata_list.get("deployment-container").value
evaluation_container = ds_model.custom_metadata_list.get("evaluation-container").value
source_name = ds_model.display_name

In [ ]:
fine_tune_output_path = "oci://<bucket_name>@<namespace>/<prefix>" # <-- Change this to the path where your pretrained fine-tuned artifact is located

### Create Model Version Set

In [ ]:
from ads.model import ModelVersionSet
from ads.model.service.oci_datascience_model_version_set import ModelVersionSetNotExists
from oci.exceptions import ServiceError
from ads.model import ModelVersionSet

mvs_name = source_name # <-- Change this to the model version set name. By default, source model name is used.

try:
    mvs = ModelVersionSet.from_name(name=mvs_name, compartment_id=compartment_id)
    mvs_id = mvs.id
    print(f"Using existing MVS {mvs_name}...")
except ModelVersionSetNotExists:
    model_version_set = (
                    ModelVersionSet()
                    .with_compartment_id(compartment_id)
                    .with_project_id(project_id)
                    .with_freeform_tags(**{"aqua_finetuning": "aqua_finetuning"})
                    .with_name(mvs_name)
                    .create()
                )
    mvs_id = model_version_set.id
except Exception as e:
    print(f"Failed to created ModelVersionSet. Exception: {e}")
    
print(f"MVS ID:{mvs_id}")

### Build Fine-Tuned Model Custom Metadata

In [ ]:
from ads.model import DataScienceModel
from ads.model.model_metadata import ModelCustomMetadata, MetadataCustomCategory
from ads.aqua.model.enums import FineTuningCustomMetadata
from ads.aqua.model.constants import ModelCustomMetadataFields

model_custom_metadata = ModelCustomMetadata()
model_custom_metadata.add(
    key=ModelCustomMetadataFields.DEPLOYMENT_CONTAINER,
    value=deployment_container,
    description="Inference container mapping for SMC",
    category=MetadataCustomCategory.OTHER,
)
model_custom_metadata.add(
    key=ModelCustomMetadataFields.EVALUATION_CONTAINER,
    value=evaluation_container,
    description="Evaluation container mapping for SMC",
    category=MetadataCustomCategory.OTHER,
)
model_custom_metadata.add(
    key=ModelCustomMetadataFields.ARTIFACT_LOCATION,
    value=source_artifact_location,
    description="artifact location",
    category=MetadataCustomCategory.OTHER,
)
model_custom_metadata.add(
    key=FineTuningCustomMetadata.FT_SOURCE_NAME,
    value=source_name,
    description="Source model name",
    category=MetadataCustomCategory.OTHER,
)
model_custom_metadata.add(
    key=FineTuningCustomMetadata.FT_SOURCE,
    value=source_model_ocid,
    description="Source model OCID",
    category=MetadataCustomCategory.OTHER,
)
model_custom_metadata.add(
    key=FineTuningCustomMetadata.FT_OUTPUT_PATH,
    value=fine_tune_output_path,
    description="Path of fine Tuned model artifacts",
    category=MetadataCustomCategory.OTHER,
)

### Add metrics for AI Quick Actions UI

The below block is optional, but if you have metrics available for your training and validation, then adding this will enable metrics report to be shown on AI Quick Actions UI. If not available, set `epochs` as 0 and the lists `train_loss`, `validation_loss`, `train_accuracy`, `validation_accuracy` to `[]`.

In [ ]:
epochs = 5 # <-- Optional, change this to the total epochs of your fine-tuning experiment.

train_loss = [0.9, 0.75, 0.6, 0.5, 0.4]  # <-- Optional, change this to training loss for each epoch.
validation_loss = [1.0, 0.85, 0.7, 0.6, 0.5] # <-- Optional, change this to validation loss for each epoch.

train_accuracy = [0.65, 0.70, 0.75, 0.8, 0.85] # <-- Optional, change this to training accuracy for each epoch.
validation_accuracy = [0.60, 0.65, 0.7, 0.75, 0.8] # <-- Optional, change this to validation accuracy for each epoch.


if not len(train_loss) == len(validation_loss) == len(train_accuracy) == len(validation_accuracy) == epochs:
    print(f"Loss and accuracy list lengths don't match with the epochs. Check the lists again.")
elif epochs > 1:
    print("Adding metrics to custom metadata...")
    for index, (t_loss, v_loss, t_acc, v_acc) in enumerate(zip(train_loss, validation_loss, train_accuracy, validation_accuracy)):
        
        model_custom_metadata.add(
            key=FineTuningCustomMetadata.TRAINING_METRICS_EPOCH.replace("epoch", f"{index+1:.2f}"),
            value=f'{{"loss": {t_loss:.4f}, "accuracy": {t_acc:.4f}, "epoch": {index+1:.1f}}}',
            description="train_metrics_epoch",
            category=MetadataCustomCategory.OTHER,
        )
        model_custom_metadata.add(
            key=FineTuningCustomMetadata.VALIDATION_METRICS_EPOCH.replace("epoch", f"{index+1:.2f}"),
            value=f'{{"loss": {v_loss:.4f}, "accuracy": {v_acc:.4f}, "epoch": {index+1:.1f}}}',
            description="val_metrics_epoch",
            category=MetadataCustomCategory.OTHER,
        )
        if index == len(train_loss)-1:
            model_custom_metadata.add(
                key=FineTuningCustomMetadata.TRAINING_METRICS_FINAL,
                value=f'{{"loss": {t_loss:.4f}, "accuracy": {t_acc:.4f}}}',
                description="train_metrics_final",
                category=MetadataCustomCategory.OTHER,
            )
            model_custom_metadata.add(
                key=FineTuningCustomMetadata.VALIDATION_METRICS_FINAL,
                value=f'{{"loss": {v_loss:.4f}, "accuracy": {v_acc:.4f}}}',
                description="val_metrics_final",
                category=MetadataCustomCategory.OTHER,
            )

### Create New FineTuned Model

In [ ]:
FT_MODEL_NAME = f"{source_name} FT Model EXT" # <-- Change this to the name you want to assign to the Fine-Tuned model entry in AI Quick Actions
FT_MODEL_DESC = f"{source_name} model Created with External Artifacts" # <-- Change this to the description of the model
MODEL_ORGANIZATION = "" # <-- Optional, change this to the organization name where the model originated.
MODEL_TASK = "text_generation" # <-- For text generation, keep it as is. Other possible values are code_synthesis, image_text_to_text, feature_extraction.

In [ ]:
model = (DataScienceModel()
        .with_compartment_id(compartment_id)
        .with_artifact(source_artifact_location, fine_tune_output_path)
        .with_project_id(project_id)
        .with_display_name(FT_MODEL_NAME)
        .with_description(FT_MODEL_DESC)
        .with_model_version_set_id(mvs_id)
        .with_freeform_tags(**{
            "OCI_AQUA": "active",
            "aqua_fine_tuned_model": f"{source_model_ocid}#{source_name}",
            "organization": MODEL_ORGANIZATION,
            "task": MODEL_TASK,
            "ready_to_fine_tune": "false",
        })
        .with_custom_metadata_list(model_custom_metadata)
        )

In [ ]:
model.create(model_by_reference=True)

Once model is created, you can visit the AI Quick Actions UI to continue with model deployment.